In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
df_coffee_review = pd.read_csv("./web_scaping/output_data_csv/coffee_bean_cleaned.csv")
df_coffee_review.head(2)

,coffee name,coffee rating,roaster name,roaster country/state,roaster city,coffee origin country,coffee origin city,roast level,whole bean (agtron),after grinding (agtron),price,pricing unit (g),review date,aroma,acidity,body,flavor,aftertaste,blind ass (paragraph),species (paragraph)
0,Colombia Pink Bourbon,95,modcup,New Jersey,['Jersey City'],Colombia,Piendamó;Cauca Department,Light,64.0,82.0,30.00,250,March 2023,9.0,9.0,9.0,9.0,9.0,"Wildly tropical, fruity and deep. Passion frui...",Produced by Wilton Benitez entirely of the Pin...
1,Kenya Kiambu Mandela Estate AA Washed Process,94,Buon Caffe,Taiwan,[' Taipei'],Kenya,Kiambu County,Medium-Light,60.0,77.0,11.52,226.8,March 2023,9.0,9.0,9.0,9.0,8.0,"Complex, nuanced, multi-layered. Black currant...",Produced by the Kariruki family from trees of ...


In [4]:
# number of rows before drop_duplicates
df_coffee_review["coffee name"].count()

1626

In [5]:
# drop dulipcates from the same roaster name and coffee name
df_coffee_review = df_coffee_review.drop_duplicates(subset= ["coffee name", "roaster name", "coffee origin country"], keep= "first")

In [6]:
# number of rows after drop_duplicates
df_coffee_review["coffee name"].count()

1568

### removing list format in roaster city

In [7]:
# remove list format in roaster city
roaster_city = []
for cell in df_coffee_review["roaster city"]:
    cell = cell[2:-2].strip()
    cell = re.sub("', '.", ";", cell)
    roaster_city.append(cell)

df_coffee_review["roaster city"] = roaster_city
df_coffee_review.head(2)

,coffee name,coffee rating,roaster name,roaster country/state,roaster city,coffee origin country,coffee origin city,roast level,whole bean (agtron),after grinding (agtron),price,pricing unit (g),review date,aroma,acidity,body,flavor,aftertaste,blind ass (paragraph),species (paragraph)
0,Colombia Pink Bourbon,95,modcup,New Jersey,Jersey City,Colombia,Piendamó;Cauca Department,Light,64.0,82.0,30.00,250,March 2023,9.0,9.0,9.0,9.0,9.0,"Wildly tropical, fruity and deep. Passion frui...",Produced by Wilton Benitez entirely of the Pin...
1,Kenya Kiambu Mandela Estate AA Washed Process,94,Buon Caffe,Taiwan,Taipei,Kenya,Kiambu County,Medium-Light,60.0,77.0,11.52,226.8,March 2023,9.0,9.0,9.0,9.0,8.0,"Complex, nuanced, multi-layered. Black currant...",Produced by the Kariruki family from trees of ...


## Identifying the coffee plant species in the species (paragraph)
---

In [8]:
df_coffee_review = df_coffee_review.dropna(subset= ["species (paragraph)"])

In [9]:
np.where(df_coffee_review["species (paragraph)"].values == np.NaN)

(array([], dtype=int64),)

In [10]:
# all arabica species name
coffee_species = ["Java", "Geisha", "Sudan Rume", "Bourbon", "Typica", "Caturra", "Villa Sarchi", "Pacas", "Venecia", "Tekisic", "SL28", 
                  "K7", "KP423", "Jackson 2/1257", "Bourbon Mayaguez 71", "Bourbon Mayaguez 139", "Mibirizi", "Maragogipe", "Nyasaland",
                  "Pache", "Harrar Rwanda", "POP3303/21", "SL14", "SL34", "Mundo Novo", "Catuai", "Pacamara", "Catimor 129", "T8667", 
                  "T5175", "Lempira", "Costa Rica 95", "Catisic", "IHCAFE 90", "Oro Azteca", "Fronton", "Anacafe 14", "Sarchimor T5296", 
                  "Cuscatleco", "Parainema", "Limani", "Obata Rojo", "IAPAR 59", "Marsellesa", "Batian", "RAB C15", "S795", "Ruiru 11", 
                  "Centroamericano", "Nayarita", "Evaluna", "Mundo Maya", "Milenio", "Starmaya", "Casiopea", "H3", "Castillo", "heirloom",
                  "Wush Wush", "wushwush", str(74110), str(74158), "Gesha", "landrace", "Robusta", "Mocca","Moka"]

In [11]:
# for loop for searching the arabica species name used in that coffee bean
species_list = []
for paragraph in df_coffee_review["species (paragraph)"]:
    row = []
    for species in coffee_species:
        if species.lower() in paragraph.lower():
            row.append(species)
    # replace Gesha to Geisha (they are the same species)
    row = list(map(lambda x: x.replace("Gesha", "Geisha"), row))
    # drop all the dup in the list
    row = [*set(row)] # set() -> return a collection of unique objects
    row_str = ";".join(row)
    species_list.append(row_str)

In [12]:
# removing species (paragraph) column
df_coffee_review = df_coffee_review.drop(columns= "species (paragraph)")
# adding a column for the arabica species
df_coffee_review["species (variety)"] = species_list

## Seperating the coffee bean dataframe into sub-dataframe
---
- Tables: Main, species, 
- create IDs: coffee_ID, country_ID, roaster_ID, species_ID, city_ID

In [13]:
# copy df_coffee_review into full_df_coffee
full_df_coffee = df_coffee_review.copy()
full_df_coffee.head(2)

,coffee name,coffee rating,roaster name,roaster country/state,roaster city,coffee origin country,coffee origin city,roast level,whole bean (agtron),after grinding (agtron),price,pricing unit (g),review date,aroma,acidity,body,flavor,aftertaste,blind ass (paragraph),species (variety)
0,Colombia Pink Bourbon,95,modcup,New Jersey,Jersey City,Colombia,Piendamó;Cauca Department,Light,64.0,82.0,30.00,250,March 2023,9.0,9.0,9.0,9.0,9.0,"Wildly tropical, fruity and deep. Passion frui...",Bourbon
1,Kenya Kiambu Mandela Estate AA Washed Process,94,Buon Caffe,Taiwan,Taipei,Kenya,Kiambu County,Medium-Light,60.0,77.0,11.52,226.8,March 2023,9.0,9.0,9.0,9.0,8.0,"Complex, nuanced, multi-layered. Black currant...",SL34;SL28


In [14]:
# create coffee ID (1 to 1568)
coffee_np = np.arange(1, 1568)

# adding coffee_ID column into df_coffee_review
full_df_coffee["coffee_ID"] = ["C" + str(x) for x in coffee_np]
full_df_coffee.head(2)

,coffee name,coffee rating,roaster name,roaster country/state,roaster city,coffee origin country,coffee origin city,roast level,whole bean (agtron),after grinding (agtron),...,pricing unit (g),review date,aroma,acidity,body,flavor,aftertaste,blind ass (paragraph),species (variety),coffee_ID
0,Colombia Pink Bourbon,95,modcup,New Jersey,Jersey City,Colombia,Piendamó;Cauca Department,Light,64.0,82.0,...,250,March 2023,9.0,9.0,9.0,9.0,9.0,"Wildly tropical, fruity and deep. Passion frui...",Bourbon,C1
1,Kenya Kiambu Mandela Estate AA Washed Process,94,Buon Caffe,Taiwan,Taipei,Kenya,Kiambu County,Medium-Light,60.0,77.0,...,226.8,March 2023,9.0,9.0,9.0,9.0,8.0,"Complex, nuanced, multi-layered. Black currant...",SL34;SL28,C2


In [15]:
full_df_coffee.columns

Index(['coffee name', 'coffee rating', 'roaster name', 'roaster country/state',
       'roaster city', 'coffee origin country', 'coffee origin city',
       'roast level', 'whole bean (agtron)', 'after grinding (agtron)',
       'price', 'pricing unit (g)', 'review date', 'aroma', 'acidity', 'body',
       'flavor', 'aftertaste', 'blind ass (paragraph)', 'species (variety)',
       'coffee_ID'],
      dtype='object')

### Origin Location Table

In [33]:
# origin table
origin_df = full_df_coffee.loc[:, ["coffee_ID", "coffee origin country"]]
origin_df.head(2)

,coffee_ID,coffee origin country
0,C1,Colombia
1,C2,Kenya


In [34]:
# spliting origin country into list
origin_df["coffee origin country"] = [n.split(";") for n in origin_df["coffee origin country"]]
# expanding the table by using explode
origin_df_expand = origin_df.explode("coffee origin country")
origin_df_expand.head()

,coffee_ID,coffee origin country
0,C1,Colombia
1,C2,Kenya
2,C3,Costa Rica
3,C4,Ethiopia
4,C5,Kenya


In [41]:
# creating a list to combine all the country in origin_df and roaster_df
country_list = list(origin_df_expand["coffee origin country"].unique())

### Roaster table

In [223]:
# roaster table
roaster_grp = full_df_coffee.loc[:, ["roaster name", "roaster city"]].groupby("roaster name")
roaster_df = pd.concat([roaster_grp["roaster city"].first()], axis= "columns", sort= False).reset_index()
roaster_df["roaster name"].count()

332

In [225]:
# generate number from 1 to 333
roaster_np = np.arange(1, 333)

# adding coffee_ID column into df_coffee_review
roaster_df["roaster_ID"] = ["R" + str(x) for x in roaster_np]
roaster_df.head(2)

,roaster name,roaster city,roaster_ID
0,94 Fresh Coffee,Kaohsiung,R1
1,A.R.C.,Hong Kong,R2


In [226]:
# spliting muliple cities and putting into a list
roaster_df["roaster city"] = [n.split(";") for n in roaster_df["roaster city"]]
# expanding the table
roaster_df = roaster_df.explode("roaster city")

#### Finding country according to city name

In [227]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(timeout = 3, user_agent="PDS") # create a geolocator object

roaster_country_list = [] # create a list to store country name

for city in roaster_df["roaster city"]:
    try:
        location = geolocator.geocode(city, language= "en") # get the location data for the city
        country = location.address.split(",")[-1].strip() # select the country name from the output
        roaster_country_list.append(country)
        print(f"{city} Done")
    except:
        roaster_country_list.append("")

roaster_df["roaster country"] = roaster_country_list

Kaohsiung Done
Hong Kong Done
Pu'Er Yunnan Province Done
Thornton Done
Milwaukee Done
Santa Rosa Beach Done
Madison Done
Hong Kong Done
Seoul Done
Fort Lauderdale Done
Douliou City Done
Salem Done
Hong Kong Done
Des Moines Done
Tainan Done
Los Angeles Done
Portland Done
Taitung Done
Arlington Done
Lee Done
Lee Done
Olympia Done
Jackson Done
Hamilton Done
Mountain View Done
Hsinchu City Done
Chicago Done
San Diego Done
Taichung Done
Taichung Done
Oakland Done
Taipei Done
Kaohsiung Done
Carbondale Done
Renton Done
Youngstown Done
Columbus Done
Taipei Done
Madison Done
Anchorage Done
Tainan Done
Taipei Done
Brooklyn Done
Manassas Done
Taipei Done
Washington Done
San Diego Done
Kaohsiung Done
Taipei Done
Santa Monica Done
Brooklyn Done
Kaohsiung Done
Cambria Done
Newtown Done
Sydney Done
Richmond Done
British Columbia Done
Taipei Done
Minneapolis Done
Annapolis Done
Taichung Done
Taipei Done
Taipei Done
Sacramento Done
San Jose Done
Montrose Done
San Luis Obispo Done
Portland Done
Kaohsiun

In [231]:
roaster_df = roaster_df[["roaster_ID", "roaster name", "roaster city", "roaster country"]]
roaster_df.head(2)

,roaster_ID,roaster name,roaster city,roaster country
0,R1,94 Fresh Coffee,Kaohsiung,Taiwan
1,R2,A.R.C.,Hong Kong,China


In [232]:
roaster_df.to_csv("./cleaned_csv/roaster.csv", encoding='utf8', index=False)

In [50]:
roaster_df = pd.read_csv("./cleaned_csv/roaster.csv")
roaster_df.head(2)

,roaster_ID,roaster name,roaster city,roaster country
0,R1,94 Fresh Coffee,Kaohsiung,Taiwan
1,R2,A.R.C.,Hong Kong,China


### Country Table

In [42]:
# counting the length of country list
print(f"Before adding roaster country: {len(country_list)}")

Before adding roaster country: 36


In [43]:
# adding roaster country that is not in the country list
for country in roaster_df["roaster country"]:
    if country not in country_list:
        country_list.append(country)
print(f"After adding roaster country: {len(country_list)}")

After adding roaster country: 46


In [44]:
# generate ID for country table
country_np = np.arange(1, 47)
country_ids = ["ctry" + str(x) for x in country_np]
len(country_ids)

46

In [65]:
# create table for country name (columns: country name and country ID)
country_df = pd.DataFrame({"country_id" : country_ids,
                           "country_name" : country_list})
country_df.head(4)

,country_id,country_name
0,ctry1,Colombia
1,ctry2,Kenya
2,ctry3,Costa Rica
3,ctry4,Ethiopia


In [74]:
# merging origin_df with the country ID and remove dup column
origin_df_merge = origin_df_expand.merge(country_df, how= "left", left_on = "coffee origin country", right_on= "country_name")\
    .drop(columns = ["coffee origin country", "country_name"])
# rename country_id column into coffee origin country ID
origin_df_cleaned = origin_df_merge.rename(columns= {"country_id" : "coffee_origin_country_id"})
origin_df_cleaned.head(2)

,coffee_ID,coffee_origin_country_id
0,C1,ctry1
1,C2,ctry2


In [80]:
# merging roaster_df with the country ID and remove dup column
roaster_df_merge = roaster_df.merge(country_df, how="left", left_on= "roaster country", right_on= "country_name")\
            .drop(columns= ["roaster country", "country_name"])
# rename columns 
roaster_df_cleaned = roaster_df_merge.rename(columns= {"roaster name" : "roaster_name",
                                                       "roaster city" : "roaster_city",
                                                       "country_id" : "roaster_country_id"})
roaster_df_cleaned.head(2)

,roaster_ID,roaster_name,roaster_city,roaster_country_id
0,R1,94 Fresh Coffee,Kaohsiung,ctry15
1,R2,A.R.C.,Hong Kong,ctry34


### Roast level table

In [104]:
roast_lv_df = full_df_coffee.loc[:,["roast level"]].groupby("roast level").first()
# there is only five different roast level -> np.range(1, 6)
roast_lv_ids = ["RL" + str(x) for x in np.arange(1, 6)]

# adding the ids into roast_lv_ids
roast_lv_df["roast_lv_id"] = roast_lv_ids
roast_lv_df = roast_lv_df.reset_index()

# rename columns
roast_lv_df_cleaned = roast_lv_df.rename(columns= {"roast level" : "roast_level"})
roast_lv_df_cleaned
 

,roast_level,roast_lv_id
0,Dark,RL1
1,Light,RL2
2,Medium,RL3
3,Medium-Dark,RL4
4,Medium-Light,RL5


### Coffee main table

In [114]:
# main table
main_df = full_df_coffee.loc[:,["coffee_ID", "coffee name", "review date", "coffee rating", "roaster name", "roast level", 
                                "whole bean (agtron)", "after grinding (agtron)", "price", "pricing unit (g)", "aroma", 
                                "acidity", "body", "flavor", "aftertaste"]]
main_df.head(2)

,coffee_ID,coffee name,review date,coffee rating,roaster name,roast level,whole bean (agtron),after grinding (agtron),price,pricing unit (g),aroma,acidity,body,flavor,aftertaste
0,C1,Colombia Pink Bourbon,March 2023,95,modcup,Light,64.0,82.0,30.00,250,9.0,9.0,9.0,9.0,9.0
1,C2,Kenya Kiambu Mandela Estate AA Washed Process,March 2023,94,Buon Caffe,Medium-Light,60.0,77.0,11.52,226.8,9.0,9.0,9.0,9.0,8.0


In [121]:
# merging the ids for roaster_df and roast level to coffee main table
main_df_merge = main_df.merge(roast_lv_df_cleaned, how= "left", left_on= "roast level", right_on= "roast_level")
main_df_merge = main_df_merge.merge(roaster_df_cleaned[["roaster_ID", "roaster_name"]], how= "left", left_on= "roaster name", right_on= "roaster_name")

# dropping the unwanted columns
main_df_drop = main_df_merge.drop(columns= ["roast level", "roast_level", "roaster name", "roaster_name"])
main_df_drop

,coffee_ID,coffee name,review date,coffee rating,whole bean (agtron),after grinding (agtron),price,pricing unit (g),aroma,acidity,body,flavor,aftertaste,roast_lv_id,roaster_ID
0,C1,Colombia Pink Bourbon,March 2023,95,64.0,82.0,30.00,250,9.0,9.0,9.0,9.0,9.0,RL2,R331
1,C2,Kenya Kiambu Mandela Estate AA Washed Process,March 2023,94,60.0,77.0,11.52,226.8,9.0,9.0,9.0,9.0,8.0,RL5,R38
2,C3,Costa Rica Volcán Azul Geisha Yeast-Washed,March 2023,94,62.0,80.0,11.52,100,9.0,9.0,9.0,9.0,8.0,RL2,R154
3,C4,Ethiopia Yirgacheffe Adame G1 Natural,March 2023,94,63.0,79.0,18.15,226.8,9.0,9.0,9.0,9.0,8.0,RL2,R56
4,C5,Kenya Gichathaini,February 2023,94,64.0,82.0,25.00,340.19,9.0,9.0,9.0,9.0,8.0,RL2,R290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1592,C1563,Costa Rica Sumava Natural Lot,December 2015,94,61.0,85.0,20.00,250,9.0,9.0,8.0,9.0,9.0,RL2,R204
1593,C1564,Ethiopia Nigusse Lemma,October 2015,94,58.0,76.0,16.95,340.19,9.0,9.0,8.0,9.0,9.0,RL5,R21
1594,C1565,Panama Altieri Natural Geisha,October 2015,96,49.0,72.0,49.95,226.8,10.0,9.0,9.0,9.0,9.0,RL3,R161
1595,C1566,Organic Espresso,October 2015,94,52.0,60.0,15.00,340.19,9.0,9.0,9.0,8.0,9.0,RL3,R47


In [ ]:
 .drop(columns= ["roast level", "roast_level"])

### Species link table

In [ ]:
# species_link table
species_df = full_df_coffee.loc[:,["coffee_ID", "species (variety)"]]
# spliting the species into list and expanding the whole table
species_df["species (variety)"] = [n.split(";") for n in species_df["species (variety)"]]
species_df = species_df.explode("species (variety)")
species_df

### Species table

In [110]:
species_ids = ["S" + str(x) for x in np.arange(1, 56)]
len(species_ids)

55

In [111]:
# reading species csv
species_full_df = pd.read_csv("./web_scaping/output_data_csv/coffee_plant_species_cleaned.csv")
species_full_df["species_id"] = species_ids

In [112]:
species_full_df.head(2)

,species name,brief intro,plant stature,leaf tip colour,bean size,optial altitude low (5N_5S),optial altitude id (5-15N_5-15S),optial altitude high (15N_15S),quality potential at high altitude,yield potential,disease: coffee leaf rust,disease: coffee berry disease,disease: nematodes,nutrition requirment,planting density,species_id
0,Anacafe 14,"Very high yielding variety, with rust resistan...",Dwarf/Compact,Green,Very Large,1200,900,700,Good,High,Resistant,Susceptible,Susceptible,High,4000-5000 a/ha (using single-stem pruning),S1
1,Batian,"A tall variety that combines high yields, tole...",Tall,Green or Bronze,Very Large,1000,700,400,Very Good,High,Tolerant,Resistant,Susceptible,Medium,2000-3000 a/ha (using multiple-stem pruning),S2


In [ ]:
df_coffee_species_cleaned

,coffee name,coffee rating,roaster name,roaster country/state,roaster city,coffee origin country,coffee origin city,roast level,whole bean (agtron),after grinding (agtron),price,pricing unit (g),review date,aroma,acidity,body,flavor,aftertaste,blind ass (paragraph),species (variety)
0,Colombia Pink Bourbon,95,modcup,New Jersey,['Jersey City'],['Colombia'],"['Piendamó', ' Cauca Department']",Light,64.0,82.0,30.00,250,March 2023,9.0,9.0,9.0,9.0,9.0,"Wildly tropical, fruity and deep. Passion frui...",[Bourbon]
1,Kenya Kiambu Mandela Estate AA Washed Process,94,Buon Caffe,Taiwan,[' Taipei'],['Kenya'],['Kiambu County'],Medium-Light,60.0,77.0,11.52,226.8,March 2023,9.0,9.0,9.0,9.0,8.0,"Complex, nuanced, multi-layered. Black currant...","[SL28, SL34]"
2,Costa Rica Volcán Azul Geisha Yeast-Washed,94,Kafe Coffee Roastery,Taiwan,['Zhubei'],['Costa Rica'],['West Valley'],Light,62.0,80.0,11.52,100,March 2023,9.0,9.0,9.0,9.0,8.0,"Richly sweet-tart, fruit-toned. Concord grape,...",[Geisha]
3,Ethiopia Yirgacheffe Adame G1 Natural,94,Caoban Coffee,Taiwan,['Taipei'],['Ethiopia'],['Yirgacheffe growing region'],Light,63.0,79.0,18.15,226.8,March 2023,9.0,9.0,9.0,9.0,8.0,"Gently sweet-tart, floral-toned. Pomegranate, ...",[]
4,Kenya Gichathaini,94,Temple Coffee,California,['Sacramento'],['Kenya'],"['Mathira West District', ' Nyeri growing regi...",Light,64.0,82.0,25.00,340.19,February 2023,9.0,9.0,9.0,9.0,8.0,"Richly sweet-savory, spice-toned. Dried fig, p...","[SL28, SL34, Ruiru 11]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1621,Costa Rica Sumava Natural Lot,94,Old Soul Co.,California,['Sacramento'],['Costa Rica'],"['Lourdes de Naranjo', ' West Valley']",Light,61.0,85.0,20.00,250,December 2015,9.0,9.0,8.0,9.0,9.0,Invitingly spicy and expansive. Candied ginger...,[]
1622,Ethiopia Nigusse Lemma,94,Barrington Coffee Roasting Co.,Massachusetts,['Lee'],['Ethiopia'],['Limu growing region'],Medium-Light,58.0,76.0,16.95,340.19,October 2015,9.0,9.0,8.0,9.0,9.0,"Subtle, elegant, alluring. Brisk, vibrant acid...",[]
1623,Panama Altieri Natural Geisha,96,Klatch Coffee,California,['Los Angeles'],['Panama'],['Boquete growing region'],Medium,49.0,72.0,49.95,226.8,October 2015,10.0,9.0,9.0,9.0,9.0,Intricate; lushly fruit-forward and round. Lil...,"[Geisha, Geisha]"
1624,Organic Espresso,94,Cafe Virtuoso,California,['San Diego'],NaN,[],Medium,52.0,60.0,15.00,340.19,October 2015,9.0,9.0,9.0,8.0,9.0,"Evaluated as espresso. Sweet-toned, round, dee...",[]


In [ ]:
flavor = 

,coffee name,coffee rating,roaster name,roaster country/state,roaster city,coffee origin country,coffee origin city,roast level,whole bean (agtron),after grinding (agtron),...,pricing unit (g),review date,aroma,acidity,body,flavor,aftertaste,blind ass (paragraph),species (paragraph),species (variety)
0,Colombia Pink Bourbon,95,modcup,New Jersey,['Jersey City'],['Colombia'],"['Piendamó', ' Cauca Department']",Light,64.0,82.0,...,250,March 2023,9.0,9.0,9.0,9.0,9.0,"Wildly tropical, fruity and deep. Passion frui...",Produced by Wilton Benitez entirely of the Pin...,[Bourbon]
1,Kenya Kiambu Mandela Estate AA Washed Process,94,Buon Caffe,Taiwan,[' Taipei'],['Kenya'],['Kiambu County'],Medium-Light,60.0,77.0,...,226.8,March 2023,9.0,9.0,9.0,9.0,8.0,"Complex, nuanced, multi-layered. Black currant...",Produced by the Kariruki family from trees of ...,"[SL28, SL34]"
2,Costa Rica Volcán Azul Geisha Yeast-Washed,94,Kafe Coffee Roastery,Taiwan,['Zhubei'],['Costa Rica'],['West Valley'],Light,62.0,80.0,...,100,March 2023,9.0,9.0,9.0,9.0,8.0,"Richly sweet-tart, fruit-toned. Concord grape,...",Produced by Alejo Castro of Volcán Azul entire...,[Geisha]
3,Ethiopia Yirgacheffe Adame G1 Natural,94,Caoban Coffee,Taiwan,['Taipei'],['Ethiopia'],['Yirgacheffe growing region'],Light,63.0,79.0,...,226.8,March 2023,9.0,9.0,9.0,9.0,8.0,"Gently sweet-tart, floral-toned. Pomegranate, ...",Produced by members of the Adame Garbota Coope...,[]
4,Kenya Gichathaini,94,Temple Coffee,California,['Sacramento'],['Kenya'],"['Mathira West District', ' Nyeri growing regi...",Light,64.0,82.0,...,340.19,February 2023,9.0,9.0,9.0,9.0,8.0,"Richly sweet-savory, spice-toned. Dried fig, p...",Produced by smallholding members of the Gikand...,"[SL28, SL34, Ruiru 11]"
